In [15]:
# install the needed packages on colab
! pip install dash jupyter-dash --quiet

In [16]:
# import pandas and plotly
 
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from jupyter_dash import JupyterDash
from dash import html, dcc 
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
from urllib.request import urlopen
# run Dash apps from within Jupyter Notebook environments
from jupyter_dash import JupyterDash
# imports
from dash import html, dcc, Dash
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [17]:
#loading the datasets
# import os
# os.chdir('/Users/chihanglung/Downloads')
df = pd.read_csv('final_house.csv')

In [18]:
# keep the first duplicated result and drop the special election
df = df.drop_duplicates(subset=['State', 'District','Candidate'], keep='first')

In [19]:
#Replace incumbent column with 'Yes' or 'No'
df['Incumbent'] = df['Incumbent'].replace({'/': 'No', 'Incumbent': 'Yes'})

In [20]:
#Total seats
df_plot = df[df['Elected'] == True]
df_plot
df_repu = df[(df['Elected'] == True) & (df['Won Party'] == 'Republician')]
df_demo = df[(df['Elected'] == True) & (df['Won Party'] == 'Democratic')]
df[df['State'] == 'Alabama']

,State,State Code,District,Party,Candidate,Incumbent,Votes,Pct%,Elected,Won Party
0,Alabama,AL,1st,Republician,Carl,Yes,139854,84.20%,True,Republician
1,Alabama,AL,1st,Libertarian Party,Remrey,No,26197,15.80%,False,Republician
2,Alabama,AL,2nd,Republician,Moore,Yes,137193,69.20%,True,Republician
3,Alabama,AL,2nd,Democratic,Harvey-Hall,No,57800,29.10%,False,Republician
4,Alabama,AL,2nd,Libertarian Party,Realz,No,3380,1.70%,False,Republician
5,Alabama,AL,3rd,Republician,Rogers,Yes,135448,71.30%,True,Republician
6,Alabama,AL,3rd,Democratic,Veasey,No,47770,25.10%,False,Republician
7,Alabama,AL,3rd,Other Party,Bell,No,3826,2.00%,False,Republician
8,Alabama,AL,3rd,Libertarian Party,Casson,No,3027,1.60%,False,Republician
9,Alabama,AL,4th,Republician,Aderholt,Yes,164371,84.20%,True,Republician


In [21]:
#creating the dataframe for map
df_grouped = df.groupby('State')

state_code = df_grouped.apply(lambda x: x['State Code'].iloc[0])
Won_party = df_grouped.apply(lambda x: x.loc[x['Elected'], 'Won Party'].value_counts().index[0])

rep_num_win_seats = df_grouped.apply(lambda x: (x['Elected'] & (x['Won Party'] == 'Republician')).sum())
demo_num_win_seats = df_grouped.apply(lambda x: (x['Elected'] & (x['Won Party'] == 'Democratic')).sum())
lib_num_win_seats = df_grouped.apply(lambda x: (x['Elected'] & (x['Won Party'] == 'Libertarian Party')).sum())

num_win_seats = df_grouped.apply(lambda x: max(
    sum(x['Elected'] & (x['Won Party'] == 'Republician')),
    sum(x['Elected'] & (x['Won Party'] == 'Democratic')),
    sum(x['Elected'] & (x['Won Party'] == 'Libertarian Party'))
))

total_seats = df_grouped['District'].nunique()

repu_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Republician', 'Votes'].sum())
demo_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Democratic', 'Votes'].sum())
liber_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Libertarian Party', 'Votes'].sum())
other_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Other Party', 'Votes'].sum())
green_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Green Party', 'Votes'].sum())
ind_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Independent Party', 'Votes'].sum())

repu_win_rate = rep_num_win_seats / total_seats

final_df = pd.DataFrame({
    'State': Won_party.index,
    'State Code': state_code.values,
    'Won Party': Won_party.values,
    'Won Seats': num_win_seats.values,
    'Total Seats': total_seats.values,
    'Republican Votes': repu_votes_total.values,
    'Democratic Votes': demo_votes_total.values,
    'Libertarian Party Votes': liber_votes_total.values,
    'Green Party Votes': green_votes_total.values,
    'Independent Party Votes': ind_votes_total.values,
    'Other Party Votes': other_votes_total.values,
    'Rep won seats %': repu_win_rate.values,
})

final_df['Rep won seats %'] = final_df['Rep won seats %'] *100
final_df

,State,State Code,Won Party,Won Seats,Total Seats,Republican Votes,Democratic Votes,Libertarian Party Votes,Green Party Votes,Independent Party Votes,Other Party Votes,Rep won seats %
0,Alabama,AL,Republician,6,7,940265,317724,74637,0,0,3826,85.714286
1,Alaska,AK,Democratic,1,1,112255,136893,0,0,0,0,0.000000
2,Arizona,AZ,Republician,6,9,934610,1004461,0,0,18851,0,66.666667
3,Arkansas,AR,Republician,4,4,596597,270900,25288,0,0,0,100.000000
4,California,CA,Democratic,40,52,3859662,6743732,0,52965,0,0,23.076923
5,Colorado,CO,Democratic,5,8,1050960,1365427,34234,0,0,21100,37.500000
6,Connecticut,CT,Democratic,5,5,526096,722267,2140,7252,4056,0,0.000000
7,Delaware,DE,Democratic,1,1,138201,178416,3074,0,0,1958,0.000000
8,Florida,FL,Republician,20,28,4270761,2904885,80331,0,0,74749,71.428571
9,Georgia,GA,Republician,9,14,2042944,1861986,0,0,0,0,64.285714


In [22]:
#datasets for bar chart1
partyvotes_df = df.groupby(['State','State Code','Party'])['Votes'].sum()
partyvotes_df = partyvotes_df.reset_index()
partyvotes_df = partyvotes_df.replace({'Libertarian Party': 'Libertarian', 'Green Party': 'Green', 'Independent Party': 'Independent', 'Other Party': 'Other'})
partyvotes_df['color'] = partyvotes_df['Party']
partyvotes_df = partyvotes_df.rename(columns={'Party':'Parties'})
partyvotes_df

,State,State Code,Parties,Votes,color
0,Alabama,AL,Democratic,317724,Democratic
1,Alabama,AL,Libertarian,74637,Libertarian
2,Alabama,AL,Other,3826,Other
3,Alabama,AL,Republician,940265,Republician
4,Alaska,AK,Democratic,136893,Democratic
...,...,...,...,...,...
159,Wisconsin,WI,Republician,1163558,Republician
160,Wyoming,WY,Democratic,47241,Democratic
161,Wyoming,WY,Libertarian,5420,Libertarian
162,Wyoming,WY,Other,4503,Other


In [23]:
#datasets for bar chart2
dff = df[df['Elected'] == True]
districtvotes_df = dff.groupby(['State','State Code','District'])['Votes'].sum()
districtvotes_df = districtvotes_df.reset_index()
districtvotes_df = pd.merge(districtvotes_df,dff[['State','District','Party','Candidate', 'Incumbent']],on=['State','District'],how='left')
districtvotes_df = districtvotes_df.drop_duplicates()
districtvotes_df

,State,State Code,District,Votes,Party,Candidate,Incumbent
0,Alabama,AL,1st,139854,Republician,Carl,Yes
1,Alabama,AL,2nd,137193,Republician,Moore,Yes
2,Alabama,AL,3rd,135448,Republician,Rogers,Yes
3,Alabama,AL,4th,164371,Republician,Aderholt,Yes
4,Alabama,AL,5th,141988,Republician,Strong,No
...,...,...,...,...,...,...,...
430,Wisconsin,WI,5th,243734,Republician,Fitzgerald,Yes
431,Wisconsin,WI,6th,0,Republician,Grothman,Yes
432,Wisconsin,WI,7th,209270,Republician,Tiffany,Yes
433,Wisconsin,WI,8th,223806,Republician,Gallagher,Yes


In [24]:

fig = px.choropleth(final_df,locations='State Code',locationmode='USA-states',color='Rep won seats %',
                    hover_name='State',hover_data=['State Code','Won Party','Won Seats',
                                                   'Total Seats','Republican Votes','Democratic Votes',
                                                   'Libertarian Party Votes','Independent Party Votes','Green Party Votes','Other Party Votes','Rep won seats %'],
                    color_continuous_scale='RdBu_r', range_color=[0,100],scope='usa',projection = 'albers usa')
fig.update_layout(height=600,width=800)

In [25]:
#extra features:
win_dff = df[df['Elected'] == True]
win_dff.loc[win_dff['State'] == 'Missouri', 'Candidate'] = 'Missouri Smith'
win_dff.loc[win_dff['State'] == 'Nebraska', 'Candidate'] = 'Nebraska Smith'
win_dff.loc[win_dff['State'] == 'New Jersey', 'Candidate'] = 'New Jersey Smith'
win_dff_grouped = win_dff.groupby('State').apply(lambda x: x.loc[x['Votes'].idxmax()])
win_dff_party = win_dff_grouped[['State','Candidate', 'Party', 'Votes']]
bar_chart3 = px.bar(win_dff_party, x='Candidate', y='Votes',hover_name='Candidate',hover_data=['State'],color= 'Party',
                    color_discrete_map={'Republician': 'rgb(178,24,43)','Democratic': 'rgb(33,102,172)'},title='The Candidate With Highest Votes In Each State')
bar_chart3

In [26]:
#Extra feature showing all candidates in the district
#datasets for bar chart4
dff4 = df
dff4 = dff4.replace({'Libertarian Party': 'Libertarian', 'Green Party': 'Green', 'Independent Party': 'Independent', 'Other Party': 'Other'})
dff4

,State,State Code,District,Party,Candidate,Incumbent,Votes,Pct%,Elected,Won Party
0,Alabama,AL,1st,Republician,Carl,Yes,139854,84.20%,True,Republician
1,Alabama,AL,1st,Libertarian,Remrey,No,26197,15.80%,False,Republician
2,Alabama,AL,2nd,Republician,Moore,Yes,137193,69.20%,True,Republician
3,Alabama,AL,2nd,Democratic,Harvey-Hall,No,57800,29.10%,False,Republician
4,Alabama,AL,2nd,Libertarian,Realz,No,3380,1.70%,False,Republician
...,...,...,...,...,...,...,...,...,...,...
1055,Wisconsin,WI,8th,Libertarian,VandenPlas,No,32043,10.50%,False,Republician
1056,Wyoming,WY,At Large,Republician,Harriet Hageman,No,132215,69.80%,True,Republician
1057,Wyoming,WY,At Large,Democratic,Lynnette GreyBull,No,47241,24.90%,False,Republician
1058,Wyoming,WY,At Large,Libertarian,Richard Brubaker,No,5420,2.90%,False,Republician


In [27]:
#program without extra features
# instantiate a JupyterDash object instead of a Dash object
app = JupyterDash(__name__)
app.layout = html.Div([html.Div([html.H1(children='US House Election 2022', id="heading"),
                       html.Hr()]),
                      html.Div([html.P(children=f'Total seats: {len(df_plot)} | Republician won: {len(df_repu)} | Democratic won: {len(df_demo)}',
                             style={'font-weight': 'bold'}),
                                dcc.Graph(id='us-election-2022-fig', figure=fig,clickData={'points': [{'customdata': ['AL']}]})],
                               style={'width': '70%', 'display': 'inline-block', 'padding': '0 20', 'float': 'left'}),
                       html.Div([html.P(id ='winning_statement',style={'font-weight': 'bold','font-size': '13px'}),dcc.Graph(id='winning-party-votes',clickData={'points': [{'customdata': ['AL']}]}),dcc.Graph(id='winning-districts-votes')],
                               style={'width': '29%', 'display': 'inline-block', 'padding': '0 20', 'float': 'right'})
                       
                        ])


@app.callback(Output('winning_statement','children'),
              Output('winning-party-votes','figure'),
              Output('winning-districts-votes','figure'),
              Input('us-election-2022-fig', 'clickData')) 
def UsElection(received):
    if not received: 
        raise PreventUpdate
    received_data = received['points'][0]['customdata'][0]
    won_parties = final_df.loc[final_df['State Code'] == received_data,'Won Party'].values[0]
    win_statement = f'Winning Party: {won_parties}'
    dff = partyvotes_df[partyvotes_df['State Code'] == received_data]
    required_state = df.loc[df['State Code'] == received_data, 'State'].values[0]
    bar_chart = px.bar(dff, x='Parties',
                         y='Votes',title= f'State: {required_state}',color= 'Parties',hover_data=['color'],
                       color_discrete_map={
        'Republician': 'rgb(178,24,43)',
        'Democratic': 'rgb(33,102,172)',
        'Libertarian': 'rgb(253,219,199)',
        'Green': 'rgb (0,128,0)',
        'Independent': 'rgb(255,255,0)',
    },category_orders={'Parties':['Republician','Democratic']})
    bar_chart.update_layout(height=300,width=500)
    dff2 = districtvotes_df[districtvotes_df['State Code'] == received_data]
    bar_chart2 = px.bar(dff2, x='District',
                         y='Votes',title=f'Number of district:{len(dff2)}',color= 'Party',color_discrete_map={
                             'Republician': 'rgb(178,24,43)',
                             'Democratic': 'rgb(33,102,172)'
                         },hover_data=['Candidate', 'Incumbent'],category_orders={'Party':['Republician','Democratic']})
    bar_chart2.update_layout(height=300,width=500)
    return win_statement,bar_chart,bar_chart2
    


app.run_server(mode='inline', port=8639)

Dash is running on http://127.0.0.1:8639/



In [28]:
#with extra features
# instantiate a JupyterDash object instead of a Dash object
app = JupyterDash(__name__)
app.layout = html.Div([html.Div([html.H1(children='US House Election 2022', id="heading"),
                       html.Hr()]),
                      html.Div([html.P(children=f'Total seats: {len(df_plot)} | Republician won: {len(df_repu)} | Democratic won: {len(df_demo)}',
                             style={'font-weight': 'bold'}),
                                dcc.Graph(id='us-election-2022-fig', figure=fig,clickData={'points': [{'customdata': ['AL']}]}),
                                dcc.Graph(id='winning_candidates',figure=bar_chart3)],
                               style={'width': '70%', 'display': 'inline-block', 'padding': '0 20', 'float': 'left'}),
                       html.Div([html.P(id ='winning_statement',style={'font-weight': 'bold','font-size': '13px'}),
                                 dcc.Graph(id='winning-party-votes'),
                                 dcc.Graph(id='winning-districts-votes',clickData={'points': [{'customdata': ['Carl']}]}),
                                 dcc.Graph(id='other-candidates')],
                               style={'width': '29%', 'display': 'inline-block', 'padding': '0 20', 'float': 'right'})
                       
                        ])


@app.callback(Output('winning_statement','children'),
              Output('winning-party-votes','figure'),
              Output('winning-districts-votes','figure'),
              Input('us-election-2022-fig', 'clickData'))
def UsElection(received):
    if not received: 
        raise PreventUpdate
    received_data = received['points'][0]['customdata'][0]
    won_parties = final_df.loc[final_df['State Code'] == received_data,'Won Party'].values[0]
    win_statement = f'Winning Party: {won_parties}'
    dff = partyvotes_df[partyvotes_df['State Code'] == received_data]
    required_state = df.loc[df['State Code'] == received_data, 'State'].values[0]
    bar_chart = px.bar(dff, x='Parties',
                         y='Votes',title= f'State: {required_state}',color= 'Parties',hover_data=['color'],
                       color_discrete_map={'Republician': 'rgb(178,24,43)',
                                           'Democratic': 'rgb(33,102,172)',
                                           'Libertarian': 'rgb(253,219,199)',
                                           'Green': 'rgb (0,128,0)',
                                           'Independent': 'rgb(255,255,0)',
                                           'Other': 'rgb(26,211,243)'},
                       category_orders={'Parties':['Republician','Democratic']})
    bar_chart.update_layout(height=300,width=500)
    dff2 = districtvotes_df[districtvotes_df['State Code'] == received_data]
    bar_chart2 = px.bar(dff2, x='District',
                         y='Votes',title=f'Number of district:{len(dff2)}',color= 'Party',color_discrete_map={
                             'Republician': 'rgb(178,24,43)',
                             'Democratic': 'rgb(33,102,172)'
                         },hover_data=['Candidate', 'Incumbent'],category_orders={'Party':['Republician','Democratic']})
    bar_chart2.update_layout(height=300,width=500)
    
    
    return win_statement,bar_chart,bar_chart2

@app.callback(Output('other-candidates','figure'),
              Input('winning-districts-votes','clickData'))     

def District_Data(clickData):
    if not clickData:
        raise PreventUpdate
    candidate = clickData['points'][0]['customdata'][0]
    district = df.loc[df['Candidate'] == candidate,'District'].values[0]
    state = df.loc[df['Candidate'] == candidate,'State'].values[0]
    dff4 = df[df['State'] == state]
    dff4 = dff4[dff4['District'] == district]
    dff4 = dff4.replace({'Libertarian Party': 'Libertarian', 'Green Party': 'Green', 'Independent Party': 'Independent', 'Other Party': 'Other'})
    bar_chart4 = px.bar(dff4, x='Candidate', y='Votes', title=f'All candidates in {district} district of {state}',
                color='Party', color_discrete_map={'Republician': 'rgb(178,24,43)',
                                                   'Democratic': 'rgb(33,102,172)',
                                                   'Libertarian': 'rgb(253,219,199)',
                                                   'Green': 'rgb (0,128,0)',
                                                   'Independent': 'rgb(255,255,0)',
                                                   'Other': 'rgb(26,211,243)'},
                hover_data=['Candidate', 'Incumbent', 'Votes', 'Pct%'], category_orders={'Party': ['Republican', 'Democratic']})
    bar_chart4.update_layout(height=300, width=500)
    return bar_chart4

app.run_server(mode='inline', port=8139)

Dash is running on http://127.0.0.1:8139/

